# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = "./WeatherPy.csv"
vacation_df = pd.read_csv(weather_data)
print(len(vacation_df))
vacation_df.head()

569


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hirara,24.8000,125.2833,80.28,89,97,8.28,JP,1666383577
1,Saint Paul Harbor,57.7900,-152.4072,49.51,46,0,17.27,US,1666383578
2,Walvis Bay,-22.9575,14.5053,61.75,82,1,9.22,NaN,1666383578
3,Turukhansk,65.8167,87.9833,33.91,98,100,18.41,RU,1666383578
4,Nikolskoye,59.7035,30.7861,42.69,93,100,3.27,RU,1666383579


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig = gmaps.figure()

#Use the Lat and Lng as locations and Humidity as the weight.
#store lat / long data
locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)

In [4]:
#set the figure format for display 
figure_layout = {
'width': '900px',
'height': '900px',
'border': '2px solid black',
'padding': '1px',
'margin': '0 auto 0 auto'
}

#create a heat map by location
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=120,
                                 point_radius = 1)

fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='2px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Drop rows that have null values
new_vacation_data = vacation_df.dropna()
#set the temerature range of the ideal location
clean_vacation_data = new_vacation_data[(new_vacation_data['Max Temp'] >= 70) 
                                        & (new_vacation_data['Max Temp'] < 80)
                                        & (new_vacation_data['Wind Speed'] < 10)
                                        & (new_vacation_data['Cloudiness'] == 0)]
print(len(clean_vacation_data))
clean_vacation_data.head()

24


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,Chhāpar,27.8167,74.4000,77.04,30,0,6.04,IN,1666383611
126,Rome,34.2570,-85.1647,73.90,28,0,3.44,US,1666383606
128,Mount Isa,-20.7333,139.5000,74.30,32,0,8.23,AU,1666383625
139,Gachsaran,30.3586,50.7981,71.69,53,0,0.00,IR,1666383629
178,Nador,35.1740,-2.9287,70.05,64,0,4.61,MA,1666383642


I decided to stick with the temperature range 70 - 80. Those are optimal temperatures for me and my family.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame()
hotel_df['City'] = ""
hotel_df['Country']= ""
hotel_df['Lat'] = ""
hotel_df['Lng'] = ""
hotel_df["Hotel Name"] = ""

#hotel_df.head()

In [7]:
#Test data used to test the code prior to running on the entire data set. 
#test_data = clean_vacation_data.head()
#test_data.head()

In [8]:
# Set the paramters
params = {
    "radius":5000,
    "type": 'hotel',
    "key": g_key    
}


# Loop through the clean_vacation_data and grab the lat/long informatin for each city

for index, row in clean_vacation_data.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]
    country = row["Country"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    params["query"] = f"Hotel in {city}, {country}"
    # Use the search term: "hotel" , radius of 5000, and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        all_hotel_names = hotels["results"][1]["name"]
        

    except Exception as e:
        
        print("did not find hotel name")    
        
    try:
        
        lat = hotels["results"][1]["geometry"]["location"]['lat']
        lng = hotels["results"][1]["geometry"]["location"]['lng']

    except Exception as e:
        
        print("did not find lat/lng")     
        
    hotel_df.loc[index, "City"] = city
    hotel_df.loc[index, "Country"] = country
    hotel_df.loc[index, "Lat"] = lat
    hotel_df.loc[index, "Lng"] = lng
    hotel_df.loc[index, "Hotel Name"] = all_hotel_names        

In [9]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
98,Chhāpar,IN,27.810582,74.434868,Tal Chhapar Forest Rest House
126,Rome,US,34.221266,-85.151218,La Quinta Inn & Suites by Wyndham Rome
128,Mount Isa,AU,-20.72595,139.49257,ibis Styles Mt Isa Verona
139,Gachsaran,IR,30.360312,50.788067,فروشگاه رفاه
178,Nador,MA,35.172245,-2.927209,Hôtel La Giralda


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
#hotel_locs = hotel_df[["Lat", "Lng"]].astype(float)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

In [12]:
# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

The figure above shows the ideal vacation locations based on the filtering conditions.